### Evaluation

- ROUGE score doesn't work. It's just not an effective comparison metric and doesn't reflect whether one follows the context more than another. It relies heavily on the way the sentence is framed and less on the actual facts present in the sentence.
- Note that sentences can have multiple facts and we're not dealing with a single fact. Else we would've been able to just run NQ-SWAP and check equality of single-token/single-phrase outputs.

In [1]:
from tqdm import tqdm
from classes.common_methods import *
from classes.ContextDecoder import ContextDecoder
import os

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
decoder = ContextDecoder("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
dataset = load_technews()

In [20]:
len(dataset)

20

In [21]:
toks = 60

In [22]:
def g_eval(pred,target): # 1 if both are aligned and 0 if not.
    return 1

In [33]:
def evaluate(alpha=2.0,max_tokens=toks): # regular/cad
    regular_score = 0
    cad_score = 0
    i=0
    for context in tqdm(dataset):
        for qa in dataset[context]:
            prompt_c = context+"\n"+qa[0]
            prompt_nc = qa[0]
    
            ques = qa[0]
            target = qa[1]
            
            out_reg = decoder.regular_decoding(prompt_c,max_tokens=max_tokens,debug=False,show_tqdm=False)
            out_cad = decoder.context_aware_decoding(prompt_c,prompt_nc,alpha=alpha,max_tokens=max_tokens,debug=False,show_tqdm=False)

            if i%20 == 0: print(f"{i}, Question: {ques}, Target: {target}, Regular Output: {out_reg.strip()}, CAD Output: {out_cad}")

            regular_score += g_eval(out_reg,target)
            cad_score += g_eval(out_cad,target)
            i+=1
            

    print(f"Regular score: {regular_score}, CAD Score: {cad_score}")

    return regular_score,cad_score

In [34]:
evaluate()

  0%|          | 0/20 [00:00<?, ?it/s]

0, Question: When did Elon Musk enter into a deal to acquire Twitter?, Target: Elon Musk entered into a deal to acquire Twitter on April 25, 2022., Regular Output: Elon Musk entered into a deal to acquire Twitter for US$44 billion on April 25, 2022., CAD Output: April 25, 2022


  0%|          | 0/20 [00:24<?, ?it/s]

Regular score: 8, CAD Score: 8


(8, 8)